In [1]:
#|default_exp img_sen2

# Sentinel 2

In [2]:
#|hide
from nbdev.showdoc import *

In [1]:
#|hide
#|export

import geopandas as gpd
import seaborn as sns
from shapely.geometry import Point
import json
import ee

ee.Authenticate()
ee.Initialize(project='ee-reutkeller')

from gee_projects import util_func as utils
from gee_projects import const_vars as const
from gee_projects import mask_s2 as mask_s2

sentinel-2 class

In [3]:
#|export

class access_s2_image():
  
  def __init__(self,
               path_to_geometry : str , # path to geometry file, can be .shp or .gpkg . example : r"D:\git\gee_maps\nbs\geometry\test_bbox.gpkg"
               crs : str , #coordinate reference system to work with
               start_date : str , # Start date for searching images. , 
               end_date : str , # End date for searching images.
               ):
    
    self.crs = crs
    self.fc = utils.gdf_to_featureCollection(
      utils.read_gdf(geometry_path=path_to_geometry,crs=self.crs))
    self.start_date=start_date
    self.end_date=end_date
    
    
    self.fc = utils.gdf_to_featureCollection(utils.read_gdf
                                        (geometry_path=path_to_geometry,crs=self.crs))
    
    s2_sr_cld_col_eval = mask_s2.get_s2_sr_cld_col(self.fc, self.start_date, self.end_date)




    

    

In [5]:
extractor = access_s2_image(path_to_geometry=r"D:\git\gee_maps\nbs\geometry\test_bbox.gpkg", 
                            crs = "4326",
                            start_date = '2023-10-06',
                            end_date = '2023-10-08')

extractor.s2_sr_cld_col_eval.getInfo()

{'type': 'ImageCollection',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535}},
  {'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535}},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535}},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535}},
  {'id': 'B5',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535}},
  {'id': 'B6',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535}},
  {'id': 'B7',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535}},
  {'id': 'B8',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535}},
  {'id': 'B8A',
   'data_type': {'type': 'PixelType',
    'precision': 'int

In [ ]:
def get_s2_sr_cld_col(fc : ee.featurecollection , # FeatureCollection with geometry datata
                      start_date : str , # Start date for searching images. , 
                      end_date : str , # End date for searching images.
                      ):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection(const.s2_img_collection_str)
        .filterBounds(fc)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte(const.CLOUDY_PIXEL_PERCENTAGE_STR, const.CLOUD_FILTER)))


    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection(const.COPERNICUS_S2_CLOUD_PROBABILITY_STR)
        .filterBounds(fc)
        .filterDate(start_date, end_date))
    
    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst(const.s2cloudless_str).apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))


